In [96]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [97]:
def getPolynomial(data, degree=1):
    if degree ==1:
        return data
    poly = PolynomialFeatures(degree= degree)
    poly_features = poly.fit_transform(data)
    return poly_features

In [98]:
pima_indians = pd.read_csv('pima_indians/pima-indians-diabetes.data.csv')

In [99]:
pima_indians.head()

,Numberoftimespregnant,Plasmaglucose,Diastolicblood,Tricepsskinfold,2-Hourserum,Bodymass,Diabetespedigree,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [100]:
# preprocessing the data
features = pima_indians.iloc[:, [1,5,7]].values
test = pima_indians.iloc[:, -1].values


# standrize the values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features[:,[0,1,2]] = scaler.fit_transform(features[:,[0,1,2]])

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
train_set, test_set, goal_train, goal_test = train_test_split(features, test, test_size = 0.2, random_state = 0)


In [101]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(train_set)
train_set = pca.transform(train_set)
test_set  = pca.transform(test_set)

In [102]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(random_state = 0)
logistic.fit(getPolynomial(train_set,3), goal_train)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(goal_test, logistic.predict(getPolynomial(test_set,3)))

print(logistic.score(getPolynomial(test_set,3), goal_test))
pd.DataFrame(cm)


0.7857142857142857


,0,1
0,94,13
1,20,27


# Cross Validation

In [107]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = logistic, X = getPolynomial(train_set,3), y = goal_train, cv = 10)
accuracies

array([0.66666667, 0.77419355, 0.80645161, 0.78688525, 0.75409836,
       0.86885246, 0.68852459, 0.67213115, 0.81967213, 0.81967213])

In [108]:
print("mean of acc:", accuracies.mean())
print("Standard diviation of acc:", accuracies.std())

mean of acc: 0.765714789353076
Standard diviation of acc: 0.06580382332195628


In [109]:
print("model acc : {:.2f} (+/- {:.2f})%".format(accuracies.mean(), accuracies.std()))

model acc : 0.77 (+/- 0.07)%


In [110]:
from sklearn.metrics import precision_score, recall_score, f1_score

y_pred = logistic.predict(getPolynomial(test_set,3))

print(precision_score(goal_test, y_pred, average='micro'))
print(recall_score(goal_test, y_pred, average='micro'))
print(f1_score(goal_test, y_pred, average='micro'))

0.7857142857142857
0.7857142857142857
0.7857142857142857


# Classification Report

In [111]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.82      0.88      0.85       107
          1       0.68      0.57      0.62        47

avg / total       0.78      0.79      0.78       154

